In [1]:
from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By
import sys, importlib

In [2]:
# importlib.reload(sys.modules['pages.elements'])
# importlib.reload(sys.modules['pages.pages'])
from scripts.inicializar import Inicializar
from pages.pages import *
from pages.elements import *

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait

In [3]:
browser = Firefox()

pagina = Inicializar(
    browser,
    'https://guardiaov4.seplag.ce.gov.br/auth',
    'credenciais.txt'
).run()

In [13]:
pagina = PageFatura(pagina.webdriver)
pagina.terceirizados.carregar_funcionarios()
f = pagina.terceirizados.funcionarios[0]

In [14]:
dados = {
    'CPF': '123',
    'Nome': 'aaa',
    'DT': 30.0,
    'SB': 1234.50,
    'ST': 4321.50,
    'Adicional': 12.0,
    'Adicional Nortuno': 22.2,
    'Reserva': 11.1,
    'Encargos': 1111.1,
    'Insalibridade': 444.1,
    'Periculosidade': 321.55,
    'Outros': 111,
    'Vale Transporte': 0.0,
    'Vale Refeicao': 400,
    'Taxa': 111,
    'Cesta': 80.0,
    'Farda': 0,
    'Municao': 0,
    'Seguro Vida': 0,
    'Supervisao': 0,
    'IJD': 0,
    'IJN': 0,
    'Tributos': 0,
    'Insumos': 0,
    'Equipamento': 0,
    'Plano Saude': 0,
    'Hora Extra': 0,
    'Hora Extra': 0,
    'DSR': 0,
    'Extra Encargos': 0, 
    'Extra Taxa': 0, 
    'Extra': 0, 
    'Viagem Tributos': 0,
    'Diarias': 0, 
    'Passagem': 0, 
    'Viagem': 0,
    'Viagem Taxa': 0, 
    'Viagem Tributos': 0
}